In [3]:
import numpy as np
#1. define categories and Questions
categories = ["Exoplanets", "Galaxies", "Constellations", "Planets", "Moons", "Astronomers/Physicists/Astronomers"]
# 5 rows = 
board_values = np.array([[100, 100, 100, 100, 100, 100], 
                         [200, 200, 200, 200, 200, 200], 
                         [300, 300, 300, 300, 300, 300], 
                         [400, 400, 400, 400, 400, 400], 
                         [500, 500, 500, 500, 500, 500] ])

# 2. Randomly pick a Daily Double location using choice
dd_selection = np.random.choice(30)

# Convert that one number into specific row (0-4) and col (0-5)
dd_row = dd_selection // 6
dd_col = dd_selection % 6

print(f"Daily Double Location: Row {dd_row}, Col {dd_col} (Category: {categories[dd_col]})")

# 3. Initialize Board with placeholders (5 rows x 6 columns)

ddArray = np.array(["Q" for _ in range(30)])
ddArray = ddArray.reshape(5, 6)
ddArray[dd_row][dd_col] = "DD" # Mark Daily Double

# 4. Display Board
print("\nJeopardy Board:")
print(ddArray)

list = []
for i in range(30):
    list.append(False)
beenUsed = np.array(list)
beenUsed = beenUsed.reshape(5, 6)
print(beenUsed)

Daily Double Location: Row 0, Col 3 (Category: Planets)

Jeopardy Board:
[['Q' 'Q' 'Q' 'D' 'Q' 'Q']
 ['Q' 'Q' 'Q' 'Q' 'Q' 'Q']
 ['Q' 'Q' 'Q' 'Q' 'Q' 'Q']
 ['Q' 'Q' 'Q' 'Q' 'Q' 'Q']
 ['Q' 'Q' 'Q' 'Q' 'Q' 'Q']]
[[False False False False False False]
 [False False False False False False]
 [False False False False False False]
 [False False False False False False]
 [False False False False False False]]


In [9]:
from clue_object_setup import Tile
from clue_object_setup import generate_tiles
from AI_Setup import create_clues
from AI_Setup import create_answers
from Play_Function import play
from AI_Setup import compare_answer
from player_object_setup import Player
from player_object_setup import startGame

answer = create_answers()
clues, answers = create_clues(answer)

board = generate_tiles(clues, answers, ddArray, board_values, beenUsed)

def assignClue(category, point_values):
    if category == "exoplanets":
        board_column = 0
    elif category == "galaxies":
        board_column = 1
    elif category == "constellations":
        board_column = 2
    elif category == "solar system":
        board_column = 3
    elif category == "moons":
        board_column = 4
    elif category == "astronomers":
        board_column = 5

    if point_values == "100":
        board_row = 0
    elif point_values == "200":
        board_row = 1
    elif point_values == "300":
        board_row = 2
    elif point_values == "400":
        board_row = 3
    elif point_values == "500":
        board_row = 4

    return board[board_row][board_column]

def getPlayerAnswer(chosenTileAnswer, chosenTileClue):
    userAnswer = input(f"{chosenTileClue}")
    output = compare_answer(chosenTileAnswer, userAnswer)
    return output

def editPoints(chosenTile, correctness):
    if correctness == "Correct":
        print("hello")
    

while True:

    startGame()
    
    

    while True:

        point_values, category = play()
        chosenTile = assignClue(category, point_values)
        if chosenTile.beenUsed:
            print("You have already selected this tile.")
            continue
        chosenTile.beenUsed = True

        print(chosenTile.clue)
        correctness = getPlayerAnswer(chosenTile.answer, chosenTile.clue)



        stop = input("Type 'Y' to stop playing:")
        if stop == "Y":
            print(f"Thanks for playing!")
            break
        elif stop == "y":
            print(f"Thanks for playing!")
            break
        else:
            continue
    
    

    break



Clues generated successfully.
HD189733b
Hello, Welcome to AstroJeopardy! Please enter user information.

Host: Hi Isabelle! Working in human rights watch sounds really impactful. What drew you to that field?

Host: That's wonderful, Isabelle. What’s something you find most rewarding about your work?

Host: I love that dedication! When you’re not working, what do you enjoy doing for fun in Columbus?

Host: Sudoku is a great brain workout! Ready to play AstroJeopardy? Type yes or no.


UnboundLocalError: cannot access local variable 'end' where it is not associated with a value

Correct
